In [16]:
import sys
sys.path.append('../../../')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Extraction from DB

## Sqlalchemy

In [17]:
import poe_price.selection as selection

In [18]:
currency = selection.get_currency()

## Psycopg2

In [19]:
from poe_price.database.psycopg2.session import PSQLSession

In [20]:
with PSQLSession('127.0.0.1', 'poe_price', 'fabio', 'password') as session:
    session.cursor.execute('SELECT * FROM trade_currency;')
    currency = session.cursor.fetchall()

In [21]:
currency = pd.DataFrame(currency, columns=['id', 'league', 'sell_currency', 'price_currency', 'sell_quantity', 'price_quantity', 'date'])

In [22]:
currency

,id,league,sell_currency,price_currency,sell_quantity,price_quantity,date
0,3905,Delirium,regret,chaos,1.0,1.0,03/28/2020
1,3906,Delirium,chaos,scour,100.0,300.0,03/28/2020
2,3907,Delirium,exa,chaos,1.0,115.0,03/28/2020
3,3908,Delirium,jew,exa,1000.0,1.0,03/28/2020
4,3909,Delirium,alt,exa,420.0,1.0,03/28/2020
...,...,...,...,...,...,...,...
9057,12990,Delirium,chaos,scour,100.0,300.0,03/28/2020
9058,12991,Delirium,exa,chaos,1.0,115.0,03/28/2020
9059,12992,Delirium,jew,exa,1000.0,1.0,03/28/2020
9060,12993,Delirium,alt,exa,420.0,1.0,03/28/2020


# Currency conversion rate

In [5]:
currency['rate'] = currency.sell_quantity / currency.price_quantity
currency.sample()

,id,league,sell_currency,price_currency,sell_quantity,price_quantity,date,rate
6703,10629,Delirium,gcp,chaos,1.0,12.0,03/29/2020,0.083333


## Test with "chaos" to "exalted" conversion (and vice-versa)

In [6]:
scurr = 'exa'
pcurr = 'chaos'

temp = currency[(currency.sell_currency==scurr) & (currency.price_currency==pcurr)].copy()
temp.sort_values('rate', ascending=False, inplace=True)

rate_std = temp.rate.std()
rate_mean = temp.rate.mean()
temp['n_rate'] = temp.rate.apply(lambda y: (y - rate_mean) / rate_std)

window = 1
f_rate = temp[(temp.n_rate > -window) & (temp.n_rate < window)].rate

np.mean(temp.head(20).rate)

0.008916611303855713

## Generalized

In [7]:
OUTLIER_WINDOW = 1    # remove trades rate which value is outside the normal distribution window 
                      # defined as (mean_value +- standard deviation * OUTLIER_WINDOW)

MARKET_HEAD = 20      # select only the top MARKET_HEAD deals for the buyer: those deals most
                      # likely are the most representative and older records are ignored

In [8]:
currencies = set(set(currency.sell_currency.values) & 
                 set(currency.price_currency.values))

c_rates = pd.DataFrame(index=set(currency.price_currency.values), 
                       columns=set(currency.price_currency.values))

In [9]:
for v1 in currencies:
    for v2 in set(currencies - set(v1)):
        
        if v1 == 'chaos' or v2 == 'chaos':
            temp = currency[(currency.sell_currency==v1) & 
                            (currency.price_currency==v2)].copy()
            temp.sort_values('rate', ascending=False, inplace=True)

            rate_std = temp.rate.std()
            rate_mean = temp.rate.mean()
            temp['n_rate'] = temp.rate.apply(lambda y: 
                                             (y - rate_mean) / rate_std)

            f_rate = temp[(temp.n_rate > -OUTLIER_WINDOW) & 
                          (temp.n_rate < OUTLIER_WINDOW)].rate

            c_rates.loc[v1, v2] = round(np.mean(temp.head(MARKET_HEAD).rate), 3)
c_rates

,alch,fuse,chrom,jew,divine,vaal,exa,blessed,chaos,chance,chisel,regret,alt,silver,gcp,regal,scour
alch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fuse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chrom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
jew,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
divine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
vaal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
exa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
blessed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
chaos,0.218,0.375,0.133,0.088,9.78,0.301,115.23,0.031,NaN,0.068,0.234,0.471,0.883,0.092,1.161,0.271,0.363
chance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
(c_rates * c_rates.T).chaos

alch        1.09174
fuse          1.161
chrom       1.04658
jew         1.00716
divine      1.05624
vaal       0.816011
exa         1.03707
blessed    0.440262
chaos           NaN
chance     0.843472
chisel      1.03475
regret     0.948123
alt         3.58145
silver     0.963792
gcp        0.918351
regal      0.890235
scour       1.10352
Name: chaos, dtype: object